<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Indicator" data-toc-modified-id="Indicator-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Indicator</a></span></li><li><span><a href="#Dataloader" data-toc-modified-id="Dataloader-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataloader</a></span></li></ul></div>

# Indicator

In [1]:
from sklearn.metrics import confusion_matrix
import numpy as np

In [2]:
class Indicator:
    def __init__(self):
        self.y_true=[]
        self.y_pred=[]
        
    def indicator_cls(self):
        tn, fp, fn, tp=confusion_matrix(self.y_true,self.y_pred ).ravel()
        POD=tp/(tp+fn)
        FAR=fp/(tp+fp)
        CSI=tp/(tp+fn+fp)
        return {'POD':POD,'FAR':FAR,'CSI':CSI}
    
    def indicator_reg(self):
        conv=np.cov([self.y_true,self.y_pred])
        CC=conv[0,1]/np.sqrt(conv[0,0]*conv[1,1])
        BIAS=np.sum(np.array(self.y_pred)-np.array(self.y_true))/np.sum(self.y_true)
        MSE=np.mean((np.array(self.y_pred)-np.array(self.y_true))**2)
        return {'CC':CC,'BIAS':BIAS,'MSE':MSE}
    
    def reset(self):
        self.y_true=[]
        self.y_pred=[]

In [3]:
indicator=Indicator()

y_true=np.array([0,1,0,1])
y_pred=np.array([0,0,0,1])
indicator.y_true.extend(y_true.tolist())
indicator.y_pred.extend(y_pred.tolist())
print(indicator.indicator_cls())
indicator.reset()

y_true=np.random.rand(10)
y_pred=np.random.rand(10)
indicator.y_true.extend(y_true.tolist())
indicator.y_pred.extend(y_pred.tolist())
print(indicator.indicator_reg())

{'POD': 0.5, 'FAR': 0.0, 'CSI': 0.5}
{'CC': 0.5219991862216652, 'BIAS': -0.16938088345983845, 'MSE': 0.08807572443283634}


# Dataloader

In [1]:
from torch.utils.data import Dataset
from datetime import date
from datetime import timedelta
import numpy as np
import random
import tqdm
import os

GOSE=np.load('/usr/commondata/weather/New/WCE/GOSE.npy',allow_pickle=True).item()
StageIV=np.load('/usr/commondata/weather/New/WCE/StageIV.npy',allow_pickle=True).item()

def date2num(start_date,end_dates):
    result=[]
    for T in end_dates:
        end_date = date(int(T[:4]), int(T[4:6]), int(T[6:8]))
        delta = (end_date - start_date)
        day=delta.days
        hour=T[8:]
        result.append('{}.{}'.format(day,hour))
    return result
    
start_date = date(2011, 12, 31)
end_dates=list(StageIV.keys())
StageIV_keys=date2num(start_date,end_dates)
for i in range(len(StageIV_keys)):
    StageIV[StageIV_keys[i]]=StageIV.pop(end_dates[i])

In [19]:
import h5py
f = h5py.File('./test.nc',mode = 'w')
for key in tqdm.tqdm(GOSE.keys()):
    f.create_dataset(key,data = GOSE[key],compression='gzip')

100%|██████████| 1376/1376 [00:25<00:00, 54.78it/s]


In [60]:
class IRDataset(Dataset):
    def __init__(self,task='identification',mode='train',balance=True):
        self.X=GOSE
        self.Y=StageIV
        
        
        if not os.path.exists('/usr/commondata/weather/New/WCE/R_samples.npy'):
            R_samples,NR_samples=self.get_samples(GOSE,StageIV)
            self.save_samples(R_samples,NR_samples)
        
        self.R_samples=np.load('/usr/commondata/weather/New/WCE/R_samples.npy')
        self.NR_samples=np.load('/usr/commondata/weather/New/WCE/NR_samples.npy')
        
        
        if task=='identification':
            R_samples=np.array(random.choices(self.R_samples,k=340000))
            NR_samples=np.array(random.choices(self.NR_samples,k=340000))
        
        if task=='estimation':
            R_samples=np.array(random.choices(self.R_samples,k=340000))
            NR_samples=np.array(random.choices(self.NR_samples,k=340000))
        
        self.samples=np.vstack([R_samples,NR_samples])
        np.random.shuffle(self.samples)
        L=len(self.samples)
        
        
        self.mode=mode
        if mode=='train':
            self.sample_idx=range(0,int(L*0.6))
        
        if mode=='test':
            self.sample_idx=range(int(L*0.6),int(L*0.8))
        
        if mode=='val':
            self.sample_idx=range(int(L*0.8),int(L*1))
        
        self.L=len(self.sample_idx)
        
        
        self.mean=np.array([407.1981814386521,905.3917506083453,1041.6140561764744]).reshape(-1,1)
        self.std_var=np.sqrt(np.array([412.5176029578715,20423.3857524064,16250.988775375401])).reshape(-1,1)

    
    @classmethod
    def crop_center(self,img,x,y,cropx,cropy):
        startx = x-(cropx)
        endx=x+(cropx)+1
        starty = y-(cropy)   
        endy= y+(cropy)+1  
        
        if len(img.shape)==3:
            _,H,W=img.shape
            if startx<0 or starty<0 or endx>=H or endy>=H:
                return None
            return img[:,startx:endx,starty:endy]
            
        if len(img.shape)==2:
            H,W=img.shape
            if startx<0 or starty<0 or endx>=H or endy>=H:
                return None
            return img[startx:endx,starty:endy]
    
    @classmethod
    def sampling(self,key,img):
        R_samples=[]
        NR_samples=[]
        for i in range(0,img.shape[0],15):
            for j in range(0,img.shape[1],15):
                Y=self.crop_center(img,i,j,14,14)
                if Y is not None:
                    if Y[14,14]>0.1:
                        R_samples.append((key,i,j))
                    else:
                        NR_samples.append((key,i,j))
        return R_samples,NR_samples
    
    @classmethod
    def get_samples(self,GOSE,StageIV):
        useful_keys=list(set(GOSE.keys())&set(StageIV.keys()))
        useful_keys=sorted(useful_keys)
        
        R_samples=[]
        NR_samples=[]
        for key in tqdm.tqdm(useful_keys):
            R_samples_tmp,NR_samples_tmp=self.sampling(key,StageIV[key])
            R_samples+=R_samples_tmp
            NR_samples+=NR_samples_tmp
        return R_samples,NR_samples
    
    @classmethod
    def save_samples(self,R_samples,NR_samples):
        R_samples=np.array(R_samples)
        NR_samples=np.array(NR_samples)
        np.save('/usr/commondata/weather/New/WCE/R_samples.npy',R_samples)
        np.save('/usr/commondata/weather/New/WCE/NR_samples.npy',NR_samples)
        

    def __getitem__(self, idx):
        key,i,j=self.samples[idx]
        X,Y=self.X[key],self.Y[key]
        i,j=int(i),int(j)
        X_croped=self.crop_center(X,i,j,14,14)
        Y_croped=Y[i,j]
        for chennel in range(3):
            X_croped[chennel,:,:]=(X_croped[chennel,:,:]-self.mean[chennel])/self.std_var[chennel]
        return X_croped,Y_croped,i,j,key


    def __len__(self):
        return self.L

    def name(self):
        return 'IRDataset'

In [61]:
dataset=IRDataset(mode='test',balance=True)

In [66]:
X_croped,Y_croped,i,j,key=dataset[4]
Y_croped

1.5649999998435

In [51]:
data=np.array(list(GOSE.values()))
data[np.isnan(data)]=0
for i in range(0,3):
    mean=np.mean(data[:,i,:,:])
    var=np.var(data[:,i,:,:])
    print(mean,var)

407.1981814386521 412.5176029578715
905.3917506083453 20423.3857524064
1041.6140561764744 16250.988775375401
